In [1]:
import pandas as pd 
import gensim
from gensim import corpora,models

# Preprocessing 

In [35]:
df = pd.read_csv('Mental-Health-Twitter.csv')
df

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,819336825231773698,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1169875706,442,230,7,1063601,0,0
19996,19996,819334654260080640,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1169875706,442,230,7,1063601,0,0
19997,19997,819334503042871297,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1169875706,442,230,7,1063601,0,0
19998,19998,819334419374899200,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1169875706,442,230,7,1063601,0,0


In [123]:
df_sample1 = df.loc[df['label'] == 1]
df_sample1

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,587768995559772160,Tue Apr 14 00:07:03 +0000 2015,"RT @CSLewisU: ""We all agreed that forgiveness ...",2766067037,78,175,664,1148,217,1
9996,9996,587719358522101760,Mon Apr 13 20:49:49 +0000 2015,Can't believe it's been a year. #blastyblast #...,2766067037,78,175,664,1148,0,1
9997,9997,587715060790734848,Mon Apr 13 20:32:44 +0000 2015,"One year ago, a journey began. I never knew th...",2766067037,78,175,664,1148,0,1
9998,9998,587065862944874497,Sun Apr 12 01:33:03 +0000 2015,I found some new hammocking friends in the par...,2766067037,78,175,664,1148,0,1


In [124]:
#convert all review text into list format
docs = df_sample1['post_text'].tolist()
docs[0]

"It's just over 2 years since I was diagnosed with #anxiety and #depression. Today I'm taking a moment to reflect on how far I've come since."

In [125]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
# [0-9][a-Z]_
# Split the documents into tokens.
# w+ indicates to match at least 1 word character
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

In [126]:
# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

In [127]:
# Remove stopwords.
stopwords = stopwords.words('english')
stopwords += ['http', 'https', 'rt', 'misslusyd', 'genevieveverso', 'thefuxedos', 'azarkansero']
stopwords += ['tauri3l', 'liz_smith333', 'ly', 'co', 'allthxngsozzie', 'bit']
docs = [[token for token in doc if token not in stopwords] for doc in docs]

In [128]:
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [129]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [130]:
# use Phrases to Compute bigrams.
from gensim.models import Phrases

# Add bigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count = 10)

In [131]:
# put the bigram (string with _) back into docs
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [132]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.

dictionary = Dictionary(docs)

# Filter out words that occur less than 10 documents, or more than 80% of the documents.
# This step would be necessary in larger text
dictionary.filter_extremes(no_below=10, no_above=0.8)

# Generate Term Document Matrix

In [133]:
# Bag-of-words representation of the documents.
# The doc2bow method is used to convert a document to its bag vector representation in the dictionary.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1300
Number of documents: 10000


In [134]:
word_counts = [[(dictionary[id], count) for id, count in line] for line in corpus]

In [135]:
# generate a unique token list 
sort_token = sorted(dictionary.items(),key=lambda k:k[0], reverse = False)

In [136]:
# unique_token
unique_token = [token for (ID,token) in sort_token]

In [137]:
import numpy as np
#gensim.matutils.corpus2dense is used to convert sparse matrix to dense matrix
matrix = gensim.matutils.corpus2dense(corpus,num_terms=len(dictionary),dtype = 'int')

matrix = matrix.T #transpose the matrix 
print(matrix)

#convert the numpy matrix into pandas data frame
matrix_df = pd.DataFrame(matrix, columns=unique_token)

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [138]:
matrix_df

,anxiety,come,depression,diagnosed,far,moment,since,taking,today,year,...,shardfox_thewitchelf,thewitchelf,battlemonkeyz,realshit,leosweare,leosweare_leo,armorogod,jackson,colt,httr
0,1,1,1,1,1,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9997,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Kmeans

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# read raw text data
docs2 = df_sample1['post_text'].tolist()

vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=(1,1))

text_tfidf = vectorizer.fit_transform(docs2)

In [140]:
n_clusters = 5  # set num of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

kmeans.fit(text_tfidf)
# Obtain the clustering results
labels = kmeans.labels_

# The clustering results are merged with the original text data
result = pd.DataFrame({'text': docs, 'cluster_label': labels})
result

,text,cluster_label
0,"[year, since, diagnosed, anxiety, depression, ...",0
1,"[sunday, need, break, planning, spend, little,...",0
2,"[awake, tired, need, sleep, brain, idea]",0
3,"[sewhq, retro, bear, make, perfect, gift, grea...",0
4,"[hard, say, whether, packing, list, making, li...",0
...,...,...
9995,"[cslewisu, agreed, forgiveness, beautiful, ide...",0
9996,"[believe, year, blastyblast, wlmhk, picstitch,...",2
9997,"[one, year, ago, journey, began, never, knew, ...",2
9998,"[found, new, hammocking, friend, park, today, ...",2


In [141]:
# Print text data for each cluster
for i in range(n_clusters):
    cluster_text = result[result['cluster_label'] == i]['text'].values
    print(f'Cluster {i + 1}:')
    for text in cluster_text:
        print(text)
    print('---------------------------------------------------------------------------------------------------------')

Cluster 1:
['year', 'since', 'diagnosed', 'anxiety', 'depression', 'today', 'taking', 'moment', 'reflect', 'far', 'come', 'since']
['sunday', 'need', 'break', 'planning', 'spend', 'little', 'time', 'possible', 'a14']
['awake', 'tired', 'need', 'sleep', 'brain', 'idea']
['sewhq', 'retro', 'bear', 'make', 'perfect', 'gift', 'great', 'beginner', 'get', 'stitching', 'october', 'sew', 'sale', 'yay']
['hard', 'say', 'whether', 'packing', 'list', 'making', 'life', 'easier', 'reinforcing', 'much', 'still', 'need', 'movinghouse', 'anxiety']
['making', 'packing', 'list', 'new', 'hobby', 'movinghouse']
['point', 'keeping', 'stuff', 'nostalgic', 'reason', 'cross', 'line', 'plain', 'old', 'hoarding', 'movinghouse']
['currently', 'finding', 'box', 'random', 'shit', 'packing', 'phase', 'think', 'closet', 'hoarder']
['bothered', 'cook', 'take', 'away', 'way', 'lazy']
['also', 'much', 'stuff', 'way', 'way', 'much', 'movinghouse']
['moving', 'stuff', 'bloomin', 'knackering', 'still', 'load']
['back', 'h

['seqilol', 'nue', 'forcing', 'look', 'dick']
['gradingus', '_fawky', 'woah', '_fawky']
['_fawky', 'thats', 'way', '_fawky']
['aartorias', 'win']
['actually', 'never', 'unless', 'change', 'never']
['halloptik', 'complaint']
['nuen_', 'ss_mangos', 'perhaps', 'mango', 'need', 'reminder', 'nuen__ss_mangos']
['gabiecake', 'make', 'exception', 'appreciator']
['gabiecake', 'nice']
['gabiecake', 'take', 'new', 'one']
['nuen_', 'ss_mangos', 'never', 'deleting', 'nuen__ss_mangos']
['_fawky', 'keep', 'inside', 'come', 'term', 'grow', 'person', '_fawky']
['_fawky', 'spiraling', 'help', 'anything', '_fawky']
['_fawky', 'excuse', '_fawky']
['_fawky', 'even', 'waste', 'energy', 'people', '_fawky']
['_fawky', 'word', 'powerful', 'careful', '_fawky']
['unknownoddity', 'good', 'dream']
['unknownoddity', 'death']
['unknownoddity', 'aside', 'nightmare', 'pretty', 'good']
['unknownoddity', 'morning', 'maddy']
['oh', 'well', 'stay', 'awake']
['body', 'letting', 'sleep']
['nightmare', 'u__u', 'u__u']
['pend

['head', 'feel', 'foggy', 'today', 'pretty', 'sure', 'bf', 'asked', 'awake', 'morning', 'answered', 'seriousness']
['people', 'use', 'phrase', 'like', 'herding', 'cat', 'describe', 'something', 'difficult', 'amp', 'futile', 'actually', 'like', 'really', 'good', 'herding', 'cat']
['jephjacques', 'propose', 'adding', 'jrgb', 'joke', 'reply', 'get', 'blocked', 'end', 'tweet', 'sincerely', 'asking', 'advice', 'information']
['people', 'pull', 'crop', 'top', 'people', 'superglued', 'crop', 'top']
['would', 'worn', 'black', 'bralet', 'strappies', 'thought', 'might', 'boobalicious', 'work', 'green', 'cami']
['feel', 'need', 'point', 'wore', 'green', 'today', 'complete', 'accident']
['hfxginger', 'girl', 'fo', 'sho']
['last', 'lb', 'short', 'short', 'amp', 'legging', 'look', 'damn', 'fine', 'thankyouverymuch']
['_spetty', 'also', 'tweet', 'came', 'woman', 'girl', 'stop', 'tearing', 'girl', 'empower', 'one', 'another', '_spetty']
['bf', 'amp', 'like', 'bee', 'gee', 'cat', 'think', 'adopted']
['

['sinus', 'hate']
['feel', 'like', 'left', 'god', 'damn', 'store', 'feel_like']
['immediately', 'regretting', 'insomnia', 'driven', 'late', 'night', 'chatter', 'ghoulettes', 'except', 'really', 'tote', 'worth', 'sleepiness']
['sleep', 'attempt', 'number', 'lessdothis']
['fall', 'asleep', 'something', 'furry', 'adorable', 'trying', 'push', 'bed']
['thing', 'passing', 'end', 'waking', 'like', 'yay', 'insomnia']
['royalpirates', 'moon', 'explaining', 'entire', 'story', 'star', 'war', 'crew', 'diagram', 'lol', 'royalpirates', 'rp', 'starwars']
['next', 'year', 'honor', 'decided', 'rock', 'paper', 'scissors']
['poor', 'poor', 'mary', 'ann', 'helped', 'woman', 'hour', 'went', 'business', 'occasional', 'giggle', 'fit', 'back']
['helped', 'indecisive', 'lady', 'pick', 'shirt', 'previously', 'decided', 'mary', 'ann', 'turn']
['bos', 'terrible', 'terrible', 'people']
['battlemonkeyz', 'also', 'tiny']
['battlemonkeyz', 'couple', 'year', 'ago', 'sister', 'made', 'snow', 'couple', 'lasted', 'day', 

['talk', 'learntolivedepressionfreeandhappy', '3uomyunvwl']
['talk', 'isyourspousedepressedwaysitispossibletohelp1', 'hgfcuzpxmd']
['talk', 'informationyoudidnotlearnaboutdepression', 'nlgykm7jdc']
['talk', 'ideastohelpyoutogetoutofthatrecession1', 'ee8evexodh']
['talk', 'whatyoucandoifyouthinkyouragingparentisdepressed', 'wpkvnikik9']
['talk', 'usethesetipstoovercomeyourmelancholy', 'u8c1ufouo4']
['experience', 'opportunity', 'patiently', 'waiting', 'happen', 'yu4zjxhjkg']
['diet', 'exercise', 'help', 'cure', 'blue', 'article', 'stuff', 'bxbyevbziv']
['step', 'step', 'straightforward', 'product', 'alternative', 'medicine', 'depression', 'article', 'stuff', 'tqlaysehjo']
['hassle', 'depression', 'treatment', 'remedy', 'described', 'article', 'stuff', '4oazxqclkv', 'depression_treatment']
['fight', 'depression', 'stay', 'happier', 'using', 'tip', 'via', 'tinamoore1234', 'newsvine', '2zjj1jmigg']
['circle', 'end', 'depression', 'therapy', 'great', 'winter', 'mental', 'health', 'ihfmjjup0

['1dupdatesonline', 'boy', 'late', 'late', 'show', 'james', 'corden', '70qii7fyjf']
['calum5sos', 'tonight', 'o9phmeuczc']
['billboard', 'hey', 'directioners', 'dream', 'come', 'true', 'control', 'trending140', 'hmg9zo3sxb', 'hodgtt']
['janetroachhw', 'everrrryyyybody', 'melbourne', 'everyone', 'rhomelbourne', 'reunionshow', 'rawessentials', 'xx', 'beuv1gq2kl']
['dean_machine_', 'anyone', 'else', 'think', 'pettifleur', 'party', 'look', 'like', 'capitol', 'rhomelbourne', '2etseaamgh', 'dean_machine_', 'look_like']
['5sos', 'buddy', 'ipye1s5wz9']
['boybanshee', 'drawing', 'janetroachhw', 'rhomelbourne', 'rerawessentials', 's9ofg9c1lm']
['boybanshee', 'drawing', 'lydiaschiavello', 'figaro', 'rhomelbourne', 'logies', 'hultccz6ty']
['daughter', 'heart', 'life', 'vf9kiuldxh']
['looking', 'unattractive', 'always', 'taz7hezckb']
['mother', 'daughter', 'mother', 'day', 'selfie', 'umai1zxim0']
['beautiful', 'daughter', 'axe6cih3xs']
['mother', 'daughter', 'selfie', 'pj1l43ps4t']
['gorgeous', 'da

['child', 'sexually', 'inappropriate', 'behavior', 'oa9smskbq5']
['ever', 'feeling', 'emptiness', 'agccxw8mhn']
['talked', 'kid', 'college', 'parenting', 'stress', 'ur2k9y1ldd']
['kid', 'toddler', 'depression', 'ccptrwyhus']
['help', 'teen', 'peer', 'pressure', 'parenting', 'x0v4hrtxdf']
['deal', 'fetal', 'alcohol', 'syndrome', 'parenting', '1zxwxyw8lp']
['vitamin', 'improve', 'sleep', 'uytgrpe2nq']
['parent', 'year', 'old', 'parenting', 'rgh6aixyac', 'year_old']
['consequence', 'making', 'threat', 'suicide', 'znjobvnwrf']
['married', 'dissatisfied', 'marriage', 'pldyaoyfcx']
['powerful', 'effect', 'gratitude', 'hmteuotku3']
['vegetarian', 'diet', 'good', 'l25vqhd7xb']
['oxytocin', 'help', 'autism', 'yirwagh2qc']
['dating', 'divorce', 'vuj8ivg90y']
['gender', 'bias', 'come', 'expressing', 'anger', 'amgcxgatfb']
['psych', 'med', 'used', 'study', 'aid', 'medication', 'ridqakotoh']
['college', 'minor', 'substance', 'abuse', 'addiction', '5samvwu7ct']
['college', 'minor', 'substance', 'abu

['tooo', 'pure']
['door', 'leave', 'locked']
['pure', 'see']
['volkswagen', 'get', 'motor', 'running']
['well', 'assume', 'dead']
['oh', 'god', 'wrong', 'u', 'tear', 'back', 'oh_god']
['yeah', 'mean', 'fatally', 'exploding']
['yup', 'see']
['oh', 'god', 'cry', 'oh_god']
['like', 'dunno', 'burn', 'dick']
['wait', 'car', 'banging']
['started', 'exhaust', 'brain', 'died']
['dunno', 'top', 'burnin', 'rubber']
['really', 'good']
['personal', 'amusement']
['oh', 'god', 'block', 'boy', 'asap', 'oh_god']
['wait', 'facebook']
['taste', 'food', 'atm', 'everything', 'meh']
['nah', 'whole', 'conversion', 'process', 'assume', 'dunno']
['nope', 'public', 'service', 'opposite', 'sin']
['yes', 'sinnamon', 'roll', 'without', 'sin']
['pure', 'innocent', 'generate', 'sin', 'idk']
['bored', 'guy']
['fun']
['tory_b_martin', 'aw', 'damn', 'always', 'miss', 'fun', 'stuff', 'tory_b_martin']
['tory_b_martin', 'miss', 'tory_b_martin']
['tskimoongoddess', 'surprised', 'still', 'one', 'piece']
['tskimoongoddess',

['xxkirbooxx', 'staystronglps', 'thank', 'much']
['everyone', 'go', 'lp', 'follow', 'wont', 'regret', 'please']
['etuetheboss', 'dud', 'gotta', 'nice', 'comeon']
['liiiiiv_', 'sgrant_', 'nice', 'get', 'everwhere', 'liiiiiv_', 'sgrant_']
['kristamarie326', 'colin_goad', 'thanks', 'wish', 'two', 'best', 'colin_goad']
['liiiiiv_', 'welcome', 'deserve', 'lt', 'liiiiiv_']
['mikebarta73', 'super', 'cool', 'dude', 'keep', 'strong', 'lt', 'strong_lt']
['liiiiiv_', 'hey', 'beautiful', 'dont', 'let', 'dumb', 'hater', 'even', 'get', 'lt', 'always', 'dont', 'judge', 'liiiiiv_']
['taylor_weldon', 'staystronglps', 'thank', 'taylor_weldon']
['taylor_weldon', 'thank', 'lt', 'hope', 'ive', 'encouraged', 'taylor_weldon']
['taylor_weldon', 'staystronglps', 'exactly', 'amp', 'make', 'great', 'amp', 'beautiful', 'person', 'taylor_weldon']
['taylor_weldon', 'knooow', 'lt', 'mean', 'nice', 'feel', 'really', 'good', 'help', 'someone', 'lt', 'hope', 'taylor_weldon']
['meganbarto', 'gurl', 'perfect', 'dont', 'l

# LDA model

In [142]:
# Train LDA model.
from gensim.models import LdaModel

In [143]:
# Set training parameters.
num_topics = 5
chunksize = 2000
#chenksize is the number of documents to be used in each training chunk.
passes = 20 #The number of model training in the whole corpus
iterations = 100 #Number of iterations of per document
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
#– Mapping from word IDs to words. 
#chunksize is 2000, passes is 20, and iterations is 100, algorithm goes through these rounds:
#Round #1: documents 0–99  
#Round #2: documents 100–199
#Round #3: documents 200-299
#Each round will iterate each document’s probability distribution assignments for a maximum of 100 times, 
#moving to the next document before 100 times if it already reached convergence.

In [144]:
lda = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)
# alpha - dictates how the topics are distributed across the documents 
# auto :  Learns an asymmetric prior from the corpus
# eta - A-priori belief on topic-word distribution, auto - Learns an asymmetric prior from the corpus.

In [145]:
# V matrix - a vector of the words
for i,topic in lda.print_topics(10):
    print(f'Top 10 words for topic #{i+1}:')
    print(topic)
    print('\n')

Top 10 words for topic #1:
0.064*"depression" + 0.024*"overcome" + 0.021*"overcome_depression" + 0.020*"health" + 0.014*"mental" + 0.013*"yes" + 0.013*"anyone" + 0.013*"anxiety" + 0.012*"family" + 0.010*"year"


Top 10 words for topic #2:
0.033*"like" + 0.025*"get" + 0.022*"one" + 0.017*"day" + 0.017*"make" + 0.016*"time" + 0.015*"know" + 0.015*"good" + 0.013*"need" + 0.012*"really"


Top 10 words for topic #3:
0.026*"shit" + 0.025*"fuck" + 0.018*"stop" + 0.017*"yeah" + 0.016*"night" + 0.015*"positive" + 0.014*"tweet" + 0.014*"thinking" + 0.014*"gt" + 0.012*"also"


Top 10 words for topic #4:
0.028*"love" + 0.027*"go" + 0.022*"life" + 0.019*"much" + 0.016*"still" + 0.016*"never" + 0.016*"say" + 0.015*"even" + 0.014*"talk" + 0.013*"someone"


Top 10 words for topic #5:
0.031*"treatment" + 0.029*"depression" + 0.022*"depression_treatment" + 0.020*"help" + 0.020*"back" + 0.019*"headache" + 0.014*"oh" + 0.013*"amp" + 0.012*"going" + 0.011*"pain"




In [148]:
import re
for i,topic in lda.print_topics(10):
    print(f'Top 10 words for topic #{i+1}:')
    print(",".join(re.findall('".*?"',topic)))# Find all strings wrapped in double quotes.
    print('\n')

Top 10 words for topic #1:
"depression","overcome","overcome_depression","health","mental","yes","anyone","anxiety","family","year"


Top 10 words for topic #2:
"like","get","one","day","make","time","know","good","need","really"


Top 10 words for topic #3:
"shit","fuck","stop","yeah","night","positive","tweet","thinking","gt","also"


Top 10 words for topic #4:
"love","go","life","much","still","never","say","even","talk","someone"


Top 10 words for topic #5:
"treatment","depression","depression_treatment","help","back","headache","oh","amp","going","pain"




In [149]:
top_topics = lda.top_topics(corpus) 

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

#pprint is suitable for the output of complex data structures, which can make the output result more neat and easy to read.
from pprint import pprint
pprint(top_topics)

Average topic coherence: -10.0441.
[([(0.032896936, 'like'),
   (0.024561316, 'get'),
   (0.021616805, 'one'),
   (0.017161418, 'day'),
   (0.016631428, 'make'),
   (0.016004797, 'time'),
   (0.01546102, 'know'),
   (0.014535891, 'good'),
   (0.012944794, 'need'),
   (0.0121975215, 'really'),
   (0.01210164, 'thing'),
   (0.012074793, 'feel'),
   (0.011792588, 'migraine'),
   (0.011702412, 'today'),
   (0.01166632, '5sos'),
   (0.011663475, 'think'),
   (0.011432775, 'got'),
   (0.011245902, 'want'),
   (0.010652123, 'see'),
   (0.010243771, 'look')],
  -4.375878936150198),
 ([(0.02757476, 'love'),
   (0.027227862, 'go'),
   (0.021984661, 'life'),
   (0.018918268, 'much'),
   (0.016368356, 'still'),
   (0.016222969, 'never'),
   (0.015928302, 'say'),
   (0.014789944, 'even'),
   (0.013681087, 'talk'),
   (0.013405287, 'someone'),
   (0.012777366, 'actually'),
   (0.012587886, 'happy'),
   (0.012054621, 'girl'),
   (0.011370877, 'everyone'),
   (0.011302241, 'hate'),
   (0.011174223, 'l

# Correlated LDA

In [154]:
pip install tomotopy

     |████████████████████████████████| 12.4 MB 257 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [155]:
import tomotopy as tp

In [156]:
# k is the number of topic
ctm = tp.CTModel(k=5)

In [157]:
#put training data into model ctm
#add document 
for doc in docs:
    ctm.add_doc(doc)

#Learning 
for i in range(0, 500, 10):
    ctm.train(10)
# 10 iterations at a time

In [158]:
U_matrix_lda_df = pd.DataFrame([doc.get_topic_dist() for doc in ctm.docs])
#get_topic_dist() Return a distribution of the topics in the document.
U_matrix_lda_df

,0,1,2,3,4
0,0.008000,0.008000,0.088000,0.328000,0.568000
1,0.010526,0.010526,0.010526,0.010526,0.957895
2,0.015385,0.015385,0.015385,0.323077,0.630769
3,0.144828,0.213793,0.075862,0.006897,0.558621
4,0.006897,0.006897,0.006897,0.075862,0.903448
...,...,...,...,...,...
9988,0.680000,0.013333,0.013333,0.146667,0.146667
9989,0.015385,0.015385,0.784615,0.015385,0.169231
9990,0.081481,0.007407,0.377778,0.007407,0.525926
9991,0.008000,0.008000,0.168000,0.008000,0.808000


In [172]:
imitate_print = lambda ctm:[(i," + ".join([str(round(p,3))+"*"+'"{}"'.format(w) 
                                           for w,p in ctm.get_topic_words(i)])) for i in range(5)]

In [173]:
ctm.get_topic_words(0)

[('shit', 0.013748862780630589),
 ('mental', 0.013176033273339272),
 ('never', 0.012125846929848194),
 ('hate', 0.009834528900682926),
 ('lt', 0.00926169939339161),
 ('every', 0.008497926406562328),
 ('something', 0.008116040378808975),
 ('better', 0.007925096899271011),
 ('pendant_kun', 0.007447738666087389),
 ('mental_health', 0.0066839661449193954)]

In [174]:
import re
for i,topic in imitate_print(ctm):
    print(f'Top 10 words for topic #{i+1}:')
    print(topic)
    print('\n')

Top 10 words for topic #1:
0.014*"shit" + 0.013*"mental" + 0.012*"never" + 0.01*"hate" + 0.009*"lt" + 0.008*"every" + 0.008*"something" + 0.008*"better" + 0.007*"pendant_kun" + 0.007*"mental_health"


Top 10 words for topic #2:
0.018*"mnwild" + 0.014*"oh" + 0.014*"game" + 0.011*"god" + 0.008*"autism_daly" + 0.008*"second" + 0.008*"done" + 0.007*"bitch" + 0.007*"as" + 0.007*"left"


Top 10 words for topic #3:
0.019*"year" + 0.013*"5sos_daily" + 0.011*"girl" + 0.01*"official_ncfc" + 0.009*"5sostumblrx" + 0.009*"michael5sos" + 0.008*"morning" + 0.008*"yes" + 0.008*"via" + 0.007*"trying"


Top 10 words for topic #4:
0.062*"depression" + 0.025*"treatment" + 0.019*"depression_treatment" + 0.015*"overcome" + 0.014*"overcome_depression" + 0.013*"help" + 0.009*"headache" + 0.009*"migraine" + 0.009*"positive" + 0.007*"thinking"


Top 10 words for topic #5:
0.015*"like" + 0.012*"get" + 0.01*"one" + 0.01*"love" + 0.009*"people" + 0.009*"know" + 0.009*"time" + 0.009*"day" + 0.008*"amp" + 0.008*"mak

In [175]:

for i,topic in imitate_print(ctm):
    print(f'Top 10 words for topic #{i+1}:')
    print(",".join(re.findall('".*?"',topic)))
    print('\n')

Top 10 words for topic #1:
"shit","mental","never","hate","lt","every","something","better","pendant_kun","mental_health"


Top 10 words for topic #2:
"mnwild","oh","game","god","autism_daly","second","done","bitch","as","left"


Top 10 words for topic #3:
"year","5sos_daily","girl","official_ncfc","5sostumblrx","michael5sos","morning","yes","via","trying"


Top 10 words for topic #4:
"depression","treatment","depression_treatment","overcome","overcome_depression","help","headache","migraine","positive","thinking"


Top 10 words for topic #5:
"like","get","one","love","people","know","time","day","amp","make"




# LSI Model

In [150]:
# review: what is corpus
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Tfidf Transformation 
#from gensim.models import LsiModel
tfidf = models.TfidfModel(corpus) #fit tfidf model
corpus_tfidf = tfidf[corpus]      #apply model to the corpus 

In [151]:
# Train LSI model

from gensim.models import LsiModel

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary.id2token, num_topics=5)
#id2word :  ID to word mapping

In [152]:
# V matrix
for i,topic in lsi.print_topics(10):
    print(f'Top 10 words for topic #{i+1}:')
    print(topic)
    print('\n')

Top 10 words for topic #1:
-0.663*"depression" + -0.471*"treatment" + -0.411*"depression_treatment" + -0.210*"overcome_depression" + -0.209*"overcome" + -0.066*"health" + -0.060*"help" + -0.056*"new" + -0.052*"time" + -0.051*"therapy"


Top 10 words for topic #2:
0.316*"love" + 0.309*"like" + 0.216*"know" + 0.210*"one" + 0.197*"get" + 0.149*"day" + -0.141*"treatment" + 0.140*"make" + 0.135*"feel" + 0.133*"good"


Top 10 words for topic #3:
-0.530*"overcome" + -0.529*"overcome_depression" + 0.442*"treatment" + 0.372*"depression_treatment" + -0.217*"depression" + 0.097*"migraine" + 0.072*"headache" + 0.045*"talk" + -0.045*"health" + -0.045*"amp"


Top 10 words for topic #4:
-0.695*"talk" + -0.431*"talk_business" + -0.431*"business" + 0.351*"love" + 0.056*"much" + -0.048*"need" + -0.034*"friend" + -0.033*"someone" + 0.032*"treatment" + 0.027*"depression_treatment"


Top 10 words for topic #5:
0.828*"love" + 0.237*"talk" + -0.186*"migraine" + 0.165*"business" + -0.164*"like" + 0.163*"talk_

In [153]:
for i,topic in lsi.print_topics(10):
    print(f'Top 10 words for topic #{i+1}:')
    print(",".join(re.findall('".*?"',topic)))
    print('\n')

Top 10 words for topic #1:
"depression","treatment","depression_treatment","overcome_depression","overcome","health","help","new","time","therapy"


Top 10 words for topic #2:
"love","like","know","one","get","day","treatment","make","feel","good"


Top 10 words for topic #3:
"overcome","overcome_depression","treatment","depression_treatment","depression","migraine","headache","talk","health","amp"


Top 10 words for topic #4:
"talk","talk_business","business","love","much","need","friend","someone","treatment","depression_treatment"


Top 10 words for topic #5:
"love","talk","migraine","business","like","talk_business","get","headache","much","time"


